In [28]:
!wget https://raw.githubusercontent.com/sirdarckcat/sirdarckcat.github.io/master/bliss/dump.json
with open('dump.json') as f:
  import json
  meaning = json.load(f)


--2021-01-05 19:04:05--  https://raw.githubusercontent.com/sirdarckcat/sirdarckcat.github.io/master/bliss/dump.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 338206 (330K) [text/plain]
Saving to: ‘dump.json.1’

dump.json.1         100%[===================>] 330.28K  --.-KB/s    in 0.02s   

2021-01-05 19:04:05 (13.1 MB/s) - ‘dump.json.1’ saved [338206/338206]



In [113]:
def define(word):
  definitions = []
  for key,value in meaning:
    if word.lower() in [k.lower() for k in key]:
      definitions.append(value)
  return definitions

In [89]:
with open('dump3.json') as f:
  import json
  known = json.load(f)

In [118]:
import re
mmm={}

def recursive_define(word, parents=None, fuzzy=True):
  key = '%s-%s'%(word,fuzzy)
  if key in mmm:
    return mmm[key]
  ret = recursive_define_raw(word, parents, fuzzy)
  mmm[key] = ret
  return ret

def recursive_define_raw(word, parents=None, fuzzy=True):
  if parents == None:
    parents = []
  if word in known:
    return ('!known', word)
  if (word.startswith('pictograph') or
      word.endswith('indicator') or
      word.startswith('symbol') or
      word.endswith('symbol') or
      word.endswith('mark') or
      word.startswith('Arabic numeral') or
      word.endswith('shape')):
    return ('!const', word)
  if fuzzy:
    fuzzy_options = [word]
    if word.startswith('equal-sided '):
      fuzzy_options.append(word.replace('equal-sided ','equilateral_'))
    if ' ' in word:
      fuzzy_options.append(word.replace(' ', '_'))
    if word.startswith('to '):
      fuzzy_options.append(word[3:])
      fuzzy_options.append(word[3:]+'-(to)')
    if word.endswith('s'):
      fuzzy_options.append(word[:-1]+'(s)')
    else:
      fuzzy_options.append(word+'s')
      fuzzy_options.append(word+'(s)')
    if '[' in word:
      fuzzy_options.append(re.sub(r'\s*\[.*','',word))
    fuzzy_results = []
    for fuzzy_option in fuzzy_options:
      fuzzy_result = recursive_define(fuzzy_option, parents, False)
      if fuzzy_result:
        fuzzy_results.append(fuzzy_result)
    if len(fuzzy_results) == 0:
      return None
    if len(fuzzy_results) == 1:
      return fuzzy_results[0]
    return ('!choose', fuzzy_results)
  definitions = define(word)
  if len(definitions) == 0 or word in parents:
    return None
  possible_definitions = []
  for definition in definitions:
    definition_symbols = []
    incomplete_definition = False
    for concept in definition:
      possibilities = re.split(r"\s*,\s*", concept)
      valid_possibilities = []
      for possibility in possibilities:
        result = recursive_define(possibility, [word]+parents)
        if result and len(result):
          valid_possibilities.append(result)
      if len(valid_possibilities) == 0:
        incomplete_definition = True
        break
      if len(valid_possibilities) == 1:
        definition_symbols.append(valid_possibilities[0])
      else:
        definition_symbols.append(('!choose', valid_possibilities))
    if incomplete_definition:
      continue
    if len(definition_symbols) == 0:
      possible_definitions.append(definition_symbols[0])
    if len(definition_symbols) == 1:
      possible_definitions.append(definition_symbols[0])
    else:
      possible_definitions.append(('!sequence', definition_symbols))
  if len(possible_definitions) == 1:
    return possible_definitions[0]
  if len(possible_definitions) == 0:
    return None
  return ('!choose', possible_definitions)


In [119]:
memory=[]
for row in meaning:
  for word in row[0]:
    memory.append((word, recursive_define(word)))

In [120]:
with open("dump4.json","w") as f:
  import json
  json.dump(memory, f)